# ILP basics

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

During the second lab of the course, we concentrate on the (I)LP modeling basics. The goal is to go through several simple examples and try to implement them using the ILP formalism.
In this document, we show the basic syntax on a simple LP problem.
Afterward, several NP-hard problems are formally presented, for which the students should fill the ILP formulations.


## Recap: Linear programming

In the linear programming, we try to solve the following problem:

$$
\min_{x} \{ c^{\text{T}} x \ | \ Ax \leq b \}, \text{ where } c \in \mathbb{R}^{n}, x \in \mathbb{R}^{n}, A \in \mathbb{R}^{m \times n}, b \in \mathbb{R}^{m}
$$

i.e., we want to minimize a linear objective function, subject to linear inequality constraints defininy the feasible region (here, convex polytope).

### Example

Take as an example the following problem:

\begin{align}
         \min_{x,y} {-x + 2y} \ \ \text{s.t.} \\
        -4x -9y \leq -18 \\
        \frac{3}{2}x - y \leq \frac{27}{4} \\
        \frac{8}{17}x - y \geq -2
\end{align}

Rewritten to the matrix form, the same problem would look as follows:

\begin{align}
            \min_{x,y} \begin{bmatrix}
            -1 & 2
            \end{bmatrix} \cdot
            \begin{bmatrix}
            x \\ y
            \end{bmatrix} \ \text{s.t.} \\
            \begin{bmatrix}
            -4 & -9 \\ \frac{3}{2} & -1 \\ -\frac{8}{17} & 1
            \end{bmatrix} \cdot
            \begin{bmatrix}
            x \\ y
            \end{bmatrix} \leq
            \begin{bmatrix}
            -18 \\ \frac{27}{4} \\ 2
            \end{bmatrix}        
\end{align}

For this simple problem, we can visualize the individual constraints, the feasible region and even the optima, which is $x = \frac{9}{2}, y = 0$.

![LP illustration](https://gitlab.com/comb-opt/course/-/raw/main/Google%20Colab%20Resources/lab02/lp.png?ref_type=heads)


Now, let us try to rewrite this problem using the Gurobi syntax. Then, we can call the solver and see if the solution that we have found 'graphically' is, indeed, correct.

The model itself will consist of variables (including definition of their domains) and constraints. Here, the implementation is pretty straightforward:

But before we start, we need to initiate gurobi.

In [1]:
!pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\trpe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Gurobi model for the example problem
import gurobipy as g  # import Gurobi

# Input data ----------------------
# The program above

# Model ---------------------------
m = g.Model()  # create an empty model
# - variables
# method addVar is used to add variable
x = m.addVar()
y = m.addVar()

# - constraints
# method addConstr is used to add constraint
m.addConstr(-4 * x -9 *y <= -18)
m.addConstr(3/2 * x -1 *y <= 27/4)
m.addConstr(8/17 * x -1 *y >= -2)

# - objective
# method setObjective is used to set objective
m.setObjective(-x + 2*y)

m.optimize()

# Solution ------------------------
print("x:", x.X)  # use [variable].X to get the optimal value
print("y:", y.X)

Set parameter Username
Set parameter LicenseID to value 2629039
Academic license - for non-commercial use only - expires 2026-02-27
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xb2d66890
Coefficient statistics:
  Matrix range     [5e-01, 9e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -4.500000000e+00
x: 4.5
y: 0.0


**While executing the model, Gurobi solver prints some information:**

- `Thread count: 2 physical cores, 4 logical processors, using up to 4 threads`

Some parts of the solver can work in parallel - which might be beneficial when solving complex models.

- `Optimize a model with 3 rows, 2 columns and 6 nonzeros`

Here, we see that the matrix $A$ contains 3 rows (constraints), 2 columns (variables) and 6 non-zero coefficients at total.

```
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
```

This parts just says that Gurobi was able to find the solution during the "Presolve" phase. For the interested reader, more about presolving in Gurobi is on webpage here: https://support.gurobi.com/hc/en-us/articles/360024738352-How-does-presolve-work

- `Optimal objective -4.500000000e+00`

The value of the objective function $c^{\text{T}} x $ is -4.5.

## Integer linear programming

Now we solve similar problem, but the domains of (some) variables are restricted to be integers.

$$
\min_{x} \{c^{T} x \ \vert \ Ax \leq b\}, c \in \mathbb{R}^{n}, x \in \mathbb{Z}^{n}, A \in \mathbb{R}^{m \times n}, b \in \mathbb{R}^{m}
$$

- Note that solving the problem as LP and rounding the solution **may not lead to optimal solution** (it might even lead to infeasible solution - see the previous example).   
- Also note that optimizing over a subset will not give better solution (it may only worsen). Therefore, the optimal objective of LP provides a bound for the optimal objective of ILP (lower bound for the minimization).

Examine the following situation and compare the optimal objectives of LP and ILP variants of the same problem:

\begin{align}
\max_{x,y} y \\ \text{ subject to} \\
-x+y \leq 1 \\
2x+3y \leq 12 \\
3x+2y \leq 12 \\
x \geq 0 \\
y \geq 0
\end{align}

![ILP vs LP](https://gitlab.com/comb-opt/course/-/raw/main/Google%20Colab%20Resources/lab02/ILP_LP.png?ref_type=heads)

The area inside of the blue polygon represents the feasible region of the LP formulation, while the red dots represent the feasible solutions of the ILP formulation.

**Now, lets enjoy some fun while implementing the models for some well-known NP-hard problems. The goal is to understand the problems and formalize them in Gurobi language.**

### Problem 1: Two partition


Given multiset $S$ of positive integers, we ask if it can be partitioned to two subsets $S_1$, $S_2$, such that sum of the numbers in $S_1$ equals to the sum of the numbers in $S_2$, and each number is assigned (either to $S_1$ or to $S_2$).

**Example 1** Multiset $S = \{ 1,1,1,2,2,3\}$ can be partitioned, e.g., to $S_1 = \{ 2, 3 \}$,  $S_2 = \{1,1,1,2\}$. Note that it is not a unique solution.

**Example 2** Multiset $S = \{2, 4\}$ cannot be partitioned.



In [9]:
import gurobipy as g

# Example input
S = [100, 50, 50, 50, 20, 20, 10]

model = g.Model()

x = model.addVars(len(S), vtype=g.GRB.BINARY)

model.addConstr(sum(x[i]*S[i] for i in range(len(S))) == sum(S)/2)

# model.setObjective(0) # it isn't necessary, they should be same 

model.optimize()

for i in range(len(S)):
  print(x[i].x)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 1 rows, 7 columns and 7 nonzeros
Model fingerprint: 0x604725db
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+01, 1e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 22 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
1.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0


### Problem 2: Maximum independent set

Given graph $G = (V,E)$, we are trying to find subset $V^{\star} \subseteq V$, such that $V^{\star}$ is as large as possible and for each pair of vertices $u,v \in V^{\star}$, $u \neq v$, it holds that there is no edge $e = \{u,v\}$ in the original graph.

In [4]:
import gurobipy as g

# Example input
n = 5  # number of vertices
edges = [(0, 1), (2, 3), (0, 4), (3, 1), (3, 4)]  # list of edges

model = g.Model()

selected = m.addVars(n)

m.addConstrs(selected[v] + selected[u] <= 1 for (v,u) in edges)

m.setObjective(selected.sum(), sense=g.GRB.MAXIMIZE)

m.optimize()

for v in range(n):
  print(selected[v].X)

# TODO: implement the model and find the solution

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 8 rows, 7 columns and 16 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+30   1.000000e+31   5.000000e+00      0s
       4    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.000000000e+00
0.0
1.0
1.0
0.0
1.0


### Problem 3: Minimum vertex cover

Given graph $G = (V,E)$, we are looking for subset $V^{\star} \subseteq V$, such that $V^{\star}$ is as small as possible and each edge $e = \{u,v\}$ is `covered' by at least one vertex $x \in V^{\star}$, i.e. $\forall e = \{u,v\} \in E \ \exists x \in V^{\star}$ such that $x = u$ or $x = v$.

In [5]:
import gurobipy as g

# Example input
n = 5  # number of vertices
edges = [(0, 1), (2, 3), (0, 4), (3, 1), (3, 4)]  # list of edges

# TODO: implement the model and find the solution

### Problem 4: SAT (boolean satisfiability problem)

Given formula $\varphi$ (in CNF), we are looking for truth assignment (True / False) to each variable, such that $\varphi$ is satisfied (evaluates to True).

**Example 1** Formula $\varphi = (x_1 \vee x_2) \wedge (\neg x_2 \vee x_3)$ can be satisfied by $x_1 := \text{True}$, $x_3 := \text{True}$, $x_2 := \text{False}$.

**Example 2** Formula $\varphi = (x_1) \wedge (\neg x_1)$ cannot be satisfied.

In [6]:
import gurobipy as g

# Example input
# SAT formula: (x0 or ~x1) and (~x0 or x1 or x2) <- encode directly into the model

# TODO: implement the model and find the solution